<a href="https://colab.research.google.com/github/ciepielajan/Multi-Class-Classification-NLP/blob/main/ML-MultinomialNB%20TfIdf%20feature%20selection%20Chi-Square.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ML-MultinomialNB TfIdf feature selection Chi-Square 

#### Pobranie danych

In [1]:
# https://drive.google.com/file/d/1fI6EXyD9TMTC1jzdu206ljXOGNjdHprq/view?usp=sharing
!gdown --id "1fI6EXyD9TMTC1jzdu206ljXOGNjdHprq"

Downloading...
From: https://drive.google.com/uc?id=1fI6EXyD9TMTC1jzdu206ljXOGNjdHprq
To: /content/user_intent.zip
100% 271k/271k [00:00<00:00, 39.7MB/s]


In [2]:
!unzip "user_intent.zip"

Archive:  user_intent.zip
  inflating: train.csv               
  inflating: __MACOSX/._train.csv    
  inflating: validation.csv          


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

data_set_train = pd.read_csv("train.csv")
data_set_valid = pd.read_csv("validation.csv")

print(data_set_train.shape)
print(data_set_valid.shape)

print(data_set_train.columns)
print(data_set_valid.columns)

(13784, 2)
(700, 2)
Index(['text', 'label'], dtype='object')
Index(['text', 'label'], dtype='object')


#### Podstawowy process text

In [4]:
import re
def process_text(sentence):
    sentence = re.sub('[A-Za-z0-9]+@[a-zA-z].[a-zA-Z]+', '', sentence)  # maile
    sentence = re.sub('(http[s]*:[/][/])[a-zA-Z0-9]+', '', sentence)  # linki
    sentence = re.sub(r"<[^>]+>", " ", sentence) # remove html tag
    sentence = re.sub(r"[^a-zA-Z\s]", "", sentence)  # remove punctuations and numbers   
    sentence = re.sub(r"\b[A-Za-z]{1}\b", "", sentence)  # remove single characters

    sentence = re.sub("^\s+|\s+$", "", sentence, flags=re.UNICODE) # Remove spaces both in the BEGINNING and in the END of a string:
    sentence = " ".join(re.split("\s+", sentence, flags=re.UNICODE))  # Remove ONLY DUPLICATE spaces:
    sentence = sentence.lower()
    
    return sentence

In [5]:
data_set_train["clean_text"] = data_set_train["text"].apply(lambda x: process_text(x))

#### `LabelEncoder` 

In [6]:
# from sklearn.preprocessing import LabelEncoder

# labelencoder = LabelEncoder()
# data_set_train["labelencoder"] = labelencoder.fit_transform(data_set_train["label"])

# # dummy_y = to_categorical(data_set_train["labelencoder"], dtype ="float32")

# data_set_train[["clean_text","label","labelencoder"]].head(5)

In [7]:
# id_intention = 6
# print("Sprawdzenie poprawności LabelEncoder i to_categorical \n")
# print("Label - ", data_set_train["label"].iloc[id_intention])
# print("LabelEncoder - ", data_set_train["labelencoder"].iloc[id_intention])
# print("return to Label - ", labelencoder.inverse_transform([ data_set_train["labelencoder"].iloc[id_intention]  ]) )

#### Zdefiniowanie X i y

In [8]:
X = data_set_train["clean_text"]
y = data_set_train["label"]

In [9]:
X.shape, y.shape

((13784,), (13784,))

#### Podział zbioru 

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((11027,), (2757,), (11027,), (2757,))

#### `TfIdfVectorizer`

W pierwszej kolejności analizie poddamy przekształcenie korpusu oczyszczonych danych za pomocą TfIdfVectorizera w celu wydobycia słownika najczęstych słów i stowrzenia macierzy cech - złożonej ze słów "kluczy" w tekstach komend poprzez przypisanie im stosownej wagi. Słowa będące najistotniejsze będą miały najwyższe wagi.

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
# stworzenie TfIdfVectorizera biorącego max 10000 najczęstszych słów ze wszystkich tekstów, zbudowanego z unigramów i bigramów
vectorizer_tf_idf = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
vectorizer_tf_idf.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=10000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

W celu wyciągnięcia najbardziej istotnych cech (z dotychczas zdefiniowanych jako max_features=10000) wykonamy następujące operacje:


1. potraktujemy każdą z kategorii jako binarną (np. 'RateBook' będzie 1 a pozostałe 0),
2. przeprowadzimy [Chi-Square](https://en.wikipedia.org/wiki/Chi-squared_test) test do oceny czy dana cecha i binarny target są niezależne,
3. pozostawimy tylko cechy z określonym p-value z Chi-Square test.



In [14]:
from sklearn import feature_selection

In [20]:
y = y_train
X_names = vectorizer_tf_idf.get_feature_names()
p_value_limit = 0.95

dtf_features = pd. DataFrame()
for cat in np.unique(y):
  chi2, p = feature_selection.chi2(X_train, y==cat)
  dtf_features = dtf_features.append(pd.DataFrame(
      {'feature': X_names, 'score': 1-p, 'y': cat}
  ))
  dtf_features = dtf_features.sort_values(['y', 'score'], ascending=[True, False])
  dtf_features = dtf_features[dtf_features['score']>p_value_limit]
X_names = dtf_features['feature'].unique().tolist()

ValueError: ignored

In [ ]:
# sprawdzenie zredukowanej liczby cech z 10000
len(X_names)

Statystycznie 5947 cech zostało uznanych za wysoce znaczące.

Wyświetlmy dla przykładu kilka z nich dla poszczególnych kategorii czynności:

In [ ]:
for cat in np.unique(y): 
  print('# {}:'.format(cat))
  print(' -> ilość wybranych cech:',
        len(dtf_features[dtf_features['y']==cat]))
  print(' -> najpopularniejsze cechy: ',', '.join(
      dtf_features[dtf_features['y']==cat]['feature'].values[:10]
  ))
  print('  ')

Zrefitujmy TfIdfVectorizer do nowego zbioru istotnych cech w celu wyprodukowania mniejszej macierzy cech i krótszego słownika.

In [ ]:
vectorizer_tf_idf = TfidfVectorizer(vocabulary=X_names)
vectorizer_tf_idf.fit(X_train)

In [ ]:
X_train = vectorizer_tf_idf.transform(X_train)
X_test = vectorizer_tf_idf.transform(X_test)

In [ ]:
# sprawdzenie rozmiaru X_train
X_train.shape, X_test.shape

In [ ]:
X_train

In [ ]:
X_train.toarray()

In [ ]:
pd.DataFrame(X_train.toarray(), columns=vectorizer_tf_idf.get_feature_names())

#### Model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score

In [ ]:
model = MultinomialNB()

# train classifier
model.fit(X_train, y_train)

# test
predicted = model.predict(X_test)
predicted_prob = model.predict_proba(X_test)

In [ ]:
predicted_prob[0]

In [ ]:
accuracy_score(y_test, predicted)

In [ ]:
confusion_matrix(y_test, predicted)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
classes = np.unique(y_test)

print('Accuracy:', round(accuracy_score(y_test, predicted),2))
print('F1_score:', round(f1_score(y_test, predicted, average='weighted'),2))

print(classification_report(y_test, predicted))
# Plot confusion matrix
cm = confusion_matrix(y_test, predicted)
fig, ax = plt.subplots()
sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap=plt.cm.Blues, cbar=False)
ax.set(xlabel='Pred', ylabel='True', xticklabels=classes, yticklabels=classes, title='Confusion matrix')
plt.yticks(rotation=0)
plt.xticks(rotation=90)

#### Predykcja na zbiorze validacyjnym

In [ ]:
# oczysczenie danych
data_set_valid["clean_text"] = data_set_valid["text"].apply(lambda x: process_text(x))

# # labelencoder 
# data_set_valid["labelencoder"] = labelencoder.fit_transform(data_set_valid["label"])

X_validate = vectorizer_tf_idf.transform(data_set_valid["clean_text"])

X_validate

In [ ]:
y_val = data_set_valid["label"].values
y_val[:5]

In [ ]:
# Sprawdzenie rozmiaru zbiorów validacyjnego
X_validate.shape, y_val.shape

In [ ]:
predicted_validate = model.predict(X_validate)
predicted_prob_validate = model.predict_proba(X_validate)

#### `Confusion matrix`

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
classes = np.unique(y_val)

print('Accuracy:', round(accuracy_score(y_val, predicted_validate),2))
print('F1_score:', round(f1_score(y_val, predicted_validate, average='weighted'),2))

print(classification_report(y_val, predicted_validate))
# Plot confusion matrix
cm = confusion_matrix(y_val, predicted_validate)
fig, ax = plt.subplots()
sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap=plt.cm.Blues, cbar=False)
ax.set(xlabel='Pred', ylabel='True', xticklabels=classes, yticklabels=classes, title='Confusion matrix')
plt.yticks(rotation=0)
plt.xticks(rotation=90)

#### Zapoznanie się z błędnymi predykcjami 

In [ ]:
indexes = []
for i, phrase in enumerate(predicted_validate):
  if phrase == 'SearchCreativeWork':
    if y_val[i] == 'SearchScreeningEvent':
      indexes.append(i)
print(indexes)

In [ ]:
# Tekst komendy użytkownika
for i in indexes:
  print(f"----------------------------\nTekst komendy:\n{data_set_valid['text'][i]}")
  print(f'True category: {y_val[i]}')
  print(f'Predicted category: {predicted_validate[i]}')
  print("\n")

Wnioski:
> 